In [1]:
import pydriller
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os 
import json

In [2]:
PATH_DATASET = "../artifacts/annotation_bugs.csv"

In [2]:
from pydriller import Repository 

In [3]:
import pathlib
from pathlib import Path

OUT_ORIGINAL_COMMIT = "../artifacts/original_commits_w_duplicates/"
#OUT_ORIGINAL_COMMIT = "../artifacts/original_commits_v02/"

In [5]:
REPO_FOLDER = "../repos"

In [6]:
df_bugs = pd.read_csv(PATH_DATASET)

In [7]:
df_bugs["only_repo_name"] = df_bugs["repo"].apply(lambda e: e.split("/")[-1])
df_bugs["msg"] = df_bugs.apply(
    lambda row: list(Repository(os.path.join(REPO_FOLDER, row["only_repo_name"]), 
                           single=row["commit_hash"]).traverse_commits())[0].msg,
    axis=1
)
df_bugs = df_bugs.sort_values(by="id")

In [8]:
df_bugs.head()

,id,real,type,repo,commit_hash,component,symptom,bug_pattern,complexity,comment,localization,only_repo_name,msg
0,1,bug,Classical,Qiskit/qiskit-ignis,ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666,Quantum State Evaluation,"Crash - OS/PL Error, Crash",Missing Error Handling,19,"From fixer: ""Currently, cvx_fit does not check...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,Fix to allow cvx_fit to retry and finally grac...
1,10,bug,Quantum,Qiskit/qiskit-ignis,e696fee3e52550f50fb240a4c6356c61c9f01725,Quantum State Evaluation,Incorrect Output,Incorrect Numerical Computation,2,"Reporter: ""ignis.characterization.calibrations...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,fix FC rotation direction (#302)
2,1010,bug,Quantum,Qiskit/qiskit-aer,dc7a2413e6c373455c00cff51e37eaaa17a4a7ac,Optimizations,"Crash - Application Error, Crash",Barrier Related,6,"Fixer: ""Currently qubit truncation will fail f...",https://github.com/Qiskit/qiskit-aer/commit/dc...,qiskit-aer,Fix ReduceBarriers to be applied before trunca...
3,1012,bug,Classical,Qiskit/qiskit-aer,7de864cc989b354d0ad1febbd62b891270927b6b,Testing,Failing Test,"Flaky Test, Misconfiguration, Misconfiguration...",1,"Fix issue where the ""diagonal"" gate is checked...",https://github.com/Qiskit/qiskit-aer/commit/7d...,qiskit-aer,Fix tolerance when checking if diagonal is uni...
4,1019,bug,Quantum,Qiskit/qiskit-aer,f3b4303f478ff84bf8da15761827287d3a2f16e4,Simulator,Incorrect Output,"API Misuse - Internal, Typo",1,"Fixer: ""The new Diagonal terra gate was being ...",https://github.com/Qiskit/qiskit-aer/commit/f3...,qiskit-aer,Fix bugs in density matrix simulation method f...


In [9]:
import re

used_human_ids = []

def get_human_id(short_repo_name, msg):
    global used_human_ids
    issue_id = msg.split()
    try:
        issue_id = re.search('#([0-9]+)', msg).group(0)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        issue_id = None # apply your error handling
    if issue_id is not None:
        new_human_id = f"{short_repo_name}{issue_id}" 
        if not new_human_id in used_human_ids:
            used_human_ids.append(new_human_id)
            return new_human_id
        return new_human_id + "_B"
    
    
df_bugs["human_id"] = df_bugs.apply(
    lambda row: get_human_id(
        short_repo_name=row["only_repo_name"], 
        msg=row["msg"]),
    axis=1
)

In [10]:
df_bugs.head()

,id,real,type,repo,commit_hash,component,symptom,bug_pattern,complexity,comment,localization,only_repo_name,msg,human_id
0,1,bug,Classical,Qiskit/qiskit-ignis,ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666,Quantum State Evaluation,"Crash - OS/PL Error, Crash",Missing Error Handling,19,"From fixer: ""Currently, cvx_fit does not check...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,Fix to allow cvx_fit to retry and finally grac...,qiskit-ignis#3
1,10,bug,Quantum,Qiskit/qiskit-ignis,e696fee3e52550f50fb240a4c6356c61c9f01725,Quantum State Evaluation,Incorrect Output,Incorrect Numerical Computation,2,"Reporter: ""ignis.characterization.calibrations...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,fix FC rotation direction (#302),qiskit-ignis#302
2,1010,bug,Quantum,Qiskit/qiskit-aer,dc7a2413e6c373455c00cff51e37eaaa17a4a7ac,Optimizations,"Crash - Application Error, Crash",Barrier Related,6,"Fixer: ""Currently qubit truncation will fail f...",https://github.com/Qiskit/qiskit-aer/commit/dc...,qiskit-aer,Fix ReduceBarriers to be applied before trunca...,qiskit-aer#693
3,1012,bug,Classical,Qiskit/qiskit-aer,7de864cc989b354d0ad1febbd62b891270927b6b,Testing,Failing Test,"Flaky Test, Misconfiguration, Misconfiguration...",1,"Fix issue where the ""diagonal"" gate is checked...",https://github.com/Qiskit/qiskit-aer/commit/7d...,qiskit-aer,Fix tolerance when checking if diagonal is uni...,qiskit-aer#707
4,1019,bug,Quantum,Qiskit/qiskit-aer,f3b4303f478ff84bf8da15761827287d3a2f16e4,Simulator,Incorrect Output,"API Misuse - Internal, Typo",1,"Fixer: ""The new Diagonal terra gate was being ...",https://github.com/Qiskit/qiskit-aer/commit/f3...,qiskit-aer,Fix bugs in density matrix simulation method f...,qiskit-aer#767


In [11]:
from collections import Counter
Counter(df_bugs["real"])

Counter({'bug': 223, 'fp': 135})

In [12]:
all_repos = list(df_bugs["repo"].unique())
all_repos

['Qiskit/qiskit-ignis',
 'Qiskit/qiskit-aer',
 'PennyLaneAI/pennylane',
 'ProjectQ-Framework/ProjectQ',
 'microsoft/QuantumLibraries',
 'microsoft/qsharp-compiler',
 'QE-Lab/OpenQL',
 'quantumlib/Cirq',
 'aws/amazon-braket-sdk-python',
 'unitaryfund/mitiq',
 'aspuru-guzik-group/tequila',
 'qulacs/qulacs',
 'xanaduai/strawberryfields',
 'Qiskit/qiskit-terra',
 'eclipse/xacc',
 'rigetti/pyquil',
 'dwavesystems/dwave-system',
 'microsoft/qsharp-runtime']

In [13]:
# trial single repository
i = 0

i_repo_long_name = all_repos[i]
i_repo_name = i_repo_long_name.split("/")[-1]
i_repo_folder = os.path.join(REPO_FOLDER, i_repo_name)

df_repo_single = df_bugs[
    df_bugs["repo"] == i_repo_long_name
]

repo_commits = list(df_repo_single["commit_hash"])

i_repo_git = Repository(i_repo_folder, only_commits=repo_commits)

In [14]:
repo_commits

['ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666',
 'e696fee3e52550f50fb240a4c6356c61c9f01725',
 '3c59f82c11e87c071bc7e84240b50e2aa995281f',
 'e8a4b14bc36dd6c1bbf11152c37e04fe597d4de7',
 '838bc0f750d63b4ea14789fd174f9a10f51f61c2',
 '640c2401b7d3821d77179a713b5ca14348a09b40',
 '13133b0e95dec69521a6b90e9dd83e17de2f6f51',
 '8f6aad2d6cf94ee33cbc8aaeb4afabad05a8cedd',
 '439b3ee739fde633337f89b617c18821b25da0ad',
 '0af27dd1fdd5c4e4d2ef80d21d0375d11a1625fd',
 'a136e72156a20c233446ebd6108038cba7ac9f1b',
 '824372e18a42a735bbef37cd8ee61bc5d5821f9f',
 '1f7b71999c28483064e91882ef1510ede23ba0e2',
 '864663d2f6212d893731ec2b5678592e4f9a4a02',
 '2f47127a99b709af2ea1f844aeee8f1d90e49194',
 '3694ca68abf54a03e73318e81d8932e6a7159f12',
 'cfcdb0db739f3497abae80898d1a9cdd2be46a37',
 '23d5ef8231d0bb95cb03ad9b1b62ee37757b24b3',
 'c4c66591c7bccc0d94bb830239611870f38a1887']

In [27]:
def modified_files_w_duplicates(modified_files):
    """Return modified files with flag if it is not unique in base filename."""
    only_filename = [mf.filename for mf in modified_files]
    duplicate_flag = [only_filename.count(fn) > 1 for fn in only_filename]
    return zip(duplicate_flag, modified_files)
    

def save_modified_file(path, modified_files):
    """Save the file at the given path if not none.\
    
    Note that this will save a before and after subfolders."""
    
    folder_before = os.path.join(path, "before")
    folder_after = os.path.join(path, "after")
    
    Path(folder_before).mkdir(parents=True, exist_ok=True)
    Path(folder_after).mkdir(parents=True, exist_ok=True)
    
    for has_duplicate, mf in modified_files_w_duplicates(modified_files):
        print("mf.filename: ", mf.filename, " - has_duplicate: ", has_duplicate)
        
        CHAR_FOR_SUBFOLDER = ">"
        
        if has_duplicate:
            out_before = mf.old_path.replace("/", CHAR_FOR_SUBFOLDER) \
                if mf.old_path is not None else "EMPTY_" + mf.new_path.replace("/", CHAR_FOR_SUBFOLDER)
            out_after = mf.new_path.replace("/", CHAR_FOR_SUBFOLDER) \
                if mf.new_path is not None else "DELETED_" + mf.old_path.replace("/", CHAR_FOR_SUBFOLDER) 
        else:
            out_before = mf.filename
            out_after = mf.filename
        
        mf_before_path = os.path.join(folder_before, out_before)
        mf_after_path = os.path.join(folder_after, out_after)  
        
        # create before version
        with open(mf_before_path, 'w') as out_file:
            content = mf.source_code_before
            content = content if content is not None else ""
            out_file.write(content)
        # after before version
        with open(mf_after_path, 'w') as out_file:
            content = mf.source_code
            content = content if content is not None else ""
            out_file.write(content)

            
def inspect_commit(pydriller_commit, commit_name=None, author_comment=None, bug_id=None):
    """Create a directory to store this commit and iterate the files."""
    c_hash = pydriller_commit.hash
    c_msg = pydriller_commit.msg
    c_project_name = pydriller_commit.project_name
    print(f"{c_project_name} => {c_hash}")
    # iterate over all modified files
    if commit_name is None: 
        commit_name = c_hash
    commit_path = os.path.join(OUT_ORIGINAL_COMMIT, c_project_name, commit_name)
    Path(commit_path).mkdir(parents=True, exist_ok=True)
    # save an identifier of the commit
    with open(os.path.join(commit_path, c_hash) + ".txt", 'w') as stub_file: 
        stub_file.write(c_msg)
        
    # save the comment resulted from the inspection
    if author_comment is not None and bug_id is not None:
        with open(os.path.join(commit_path, f"bug_{bug_id}_comment.md"), 'w') as stub_file: 
            stub_file.write(author_comment)
    
    # save a metadata file
    metadata = {
        "commit_hash": c_hash,
        "commit_msg": c_msg,
        "project_name": c_project_name,
        "id": bug_id,
        "human_id": commit_name,
        "author_comment": author_comment
    }
    with open(os.path.join(commit_path, "metadata.json"), "w") as meta_file:
        json.dump(metadata, meta_file)
    
    
    save_modified_file(path=commit_path, modified_files=list(pydriller_commit.modified_files))
    return commit_path

#for c in i_repo_git.traverse_commits():
#    inspect_commit(c)
         

In [28]:
df_real_bugs = df_bugs[
    df_bugs["real"] == "bug"
]

# CREATE FOLDER STRUCTURE
Path(OUT_ORIGINAL_COMMIT).mkdir(parents=True, exist_ok=True)


df_real_bugs["local_file_path"] = df_real_bugs.apply(
    lambda row: inspect_commit(
        pydriller_commit=list(
            Repository(
                os.path.join(REPO_FOLDER, row["only_repo_name"]),
                single=row["commit_hash"]
            ).traverse_commits()
        )[0],
        commit_name=row["human_id"],
        author_comment=row["comment"],
        bug_id=row["id"]
    ),
    axis=1
)

qiskit-ignis => ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666
mf.filename:  cvx_fit.py  - has_duplicate:  False
qiskit-ignis => e696fee3e52550f50fb240a4c6356c61c9f01725
mf.filename:  ibmq_utils.py  - has_duplicate:  False
qiskit-aer => dc7a2413e6c373455c00cff51e37eaaa17a4a7ac
mf.filename:  controller.hpp  - has_duplicate:  False
mf.filename:  qasm_controller.hpp  - has_duplicate:  False
qiskit-aer => 7de864cc989b354d0ad1febbd62b891270927b6b
mf.filename:  fix-diagonal-unitary-check-a726500ffda6c74b.yaml  - has_duplicate:  False
mf.filename:  operations.hpp  - has_duplicate:  False
mf.filename:  qasm_unitary_gate.py  - has_duplicate:  False
mf.filename:  ref_diagonal_gate.py  - has_duplicate:  False
qiskit-aer => f3b4303f478ff84bf8da15761827287d3a2f16e4
mf.filename:  fix-density-matrix-diagonal-bb8ce049e2fa3c64.yaml  - has_duplicate:  False
mf.filename:  densitymatrix_state.hpp  - has_duplicate:  False
qiskit-aer => 0eab74b57f2704ad0893132b6c5f214603550c39
mf.filename:  densitymatrix_thrust.h

mf.filename:  SyntaxExtensions.fs  - has_duplicate:  False
qsharp-compiler => 7173815bb77d20b7cc7a342e8569b77b5e79e612
mf.filename:  CompilationLoader.cs  - has_duplicate:  False
qsharp-compiler => 1ca68a9c43a9d6c05f7fb4cb1ce46295e70d6091
mf.filename:  Library1.csproj  - has_duplicate:  False
mf.filename:  Example.csproj  - has_duplicate:  False
qsharp-compiler => 34a8c0b0903005d5d9a89b67ef0deec65cb6f0fd
mf.filename:  build.ps1  - has_duplicate:  False
mf.filename:  pack.ps1  - has_duplicate:  False
mf.filename:  test.ps1  - has_duplicate:  False
qsharp-compiler => b1c3ec68f4bfae42eaaa3e2026f077332cccfa04
mf.filename:  manifest.ps1  - has_duplicate:  False
mf.filename:  pack-extensions.ps1  - has_duplicate:  False
mf.filename:  pack.ps1  - has_duplicate:  False
qsharp-compiler => cb553b73f02833f7e9c74fa83ca787b1c0a7e603
mf.filename:  ClassicalControlTests.fs  - has_duplicate:  False
mf.filename:  ClassicalControl.qs  - has_duplicate:  False
mf.filename:  Signatures.fs  - has_duplicate:

mf.filename:  CHANGELOG.md  - has_duplicate:  False
mf.filename:  tape.py  - has_duplicate:  False
mf.filename:  amplitude.py  - has_duplicate:  False
mf.filename:  particle_conserving_u1.py  - has_duplicate:  False
mf.filename:  particle_conserving_u2.py  - has_duplicate:  False
mf.filename:  adjoint.py  - has_duplicate:  False
mf.filename:  utils.py  - has_duplicate:  False
mf.filename:  test_default_qubit.py  - has_duplicate:  False
mf.filename:  test_tape.py  - has_duplicate:  False
mf.filename:  test_particle_conserving_u1.py  - has_duplicate:  False
mf.filename:  test_particle_conserving_u2.py  - has_duplicate:  False
mf.filename:  test_utils.py  - has_duplicate:  False
mf.filename:  test_adjoint.py  - has_duplicate:  False
Cirq => a3ab283cecf50c9e74e6f5a586e07f63e55cac8a
mf.filename:  decompositions_test.py  - has_duplicate:  False
Cirq => 4dae9b2c766fb9cd5a24bce655020b63c0ed47ac
mf.filename:  n_qubit_tomography.py  - has_duplicate:  False
mf.filename:  n_qubit_tomography_test.p

mitiq => 4d260bd6545c20f3b1fefc2c843fb67d1296cb45
mf.filename:  folding.py  - has_duplicate:  False
mf.filename:  test_folding.py  - has_duplicate:  False
mitiq => 6f66672740d1322995916dbeb1ffc2f94403baac
mf.filename:  CHANGELOG.md  - has_duplicate:  False
mf.filename:  _about.py  - has_duplicate:  False
tequila => d08a1ab5af20263cd2985036ac14e2ed127894a3
tequila => fd69d263e63ab8a7e05dfcb2776d486f2b964d07
tequila => 30eba3a9bfcdf2062a2f7808c520d1a7c4bd1236
tequila => 7f5a90d27de533c48637a0399f482390b7d6c0a1
tequila => 4c08588ab73c8a23537596a728a4b8f489ec4048
tequila => 2790f8ee8bc6564a3482d0d2176c743ab5b9e3d7
tequila => 27ac6d02a4f7020a1fb2a2c5fa8562d77d1662fd
tequila => 69c3d3fa43f4ebe79cc73cef79f3ee18677e2344
tequila => 2fab2353ed96acdc80bdc65eccac43031b98180b
tequila => a67fe808c165c5f30a30d9745cfb8155b47c7eca
mf.filename:  qubit_wavefunction.py  - has_duplicate:  False
mf.filename:  test_simulator_backends.py  - has_duplicate:  False
tequila => be70211e1b4cad4326f01f9b2f790643255a

dwave-system => 91a5b5ccd746906b701fe22adca0bd5716f79a47
dwave-system => ff44e1bbad1fce93a4685d65119e1b6d96cf23b2
dwave-system => 00fa6703c7e550314e5ea171ca4c10d15282079c
dwave-system => abce195feee853ed9a82694af72c3fb59818bb4e
qsharp-runtime => c245ecbc908adfa830c19be4dec5c50f58a07837
mf.filename:  QArray.cs  - has_duplicate:  False
mf.filename:  Range.cs  - has_duplicate:  False
qsharp-runtime => e1d4c07c299b31d3486647e0ce3748df97a0d786
mf.filename:  SimulationCode.fs  - has_duplicate:  False
mf.filename:  Issue76.qs  - has_duplicate:  False
qsharp-runtime => 00818415e46a90d880f4b4cc11a4f89ab88ff8ff
mf.filename:  ToffoliSimulatorTests.cs  - has_duplicate:  False
qsharp-runtime => eaafd1ba3e4e421655889bdf2f5d0daa18ae96f2
qsharp-runtime => f4a4a6d959dc0e97bc8dc12fc67ddbe78423f5fe
mf.filename:  QubitManager.cs  - has_duplicate:  False
mf.filename:  QubitManagerTests.cs  - has_duplicate:  False
qsharp-runtime => 4a5862ffae68159aa0918ec9ed7f955d30a1ade3
mf.filename:  CodegenTests.qs  - ha

qiskit-ignis => 439b3ee739fde633337f89b617c18821b25da0ad
mf.filename:  pulse_schedules.py  - has_duplicate:  False
mf.filename:  fix-meas-pulse-alignment-drag-0660aeb9609b00a3.yaml  - has_duplicate:  False
qiskit-ignis => 0af27dd1fdd5c4e4d2ef80d21d0375d11a1625fd
mf.filename:  circuits.py  - has_duplicate:  False
mf.filename:  rb-seed-fix-24957ed1e9ffe31c.yaml  - has_duplicate:  False
mf.filename:  generate_data.py  - has_duplicate:  False
mf.filename:  test_fitter_cnotdihedral_X_results.json  - has_duplicate:  False
mf.filename:  test_fitter_cnotdihedral_Z_results.json  - has_duplicate:  False
mf.filename:  test_fitter_cnotdihedral_expected_results.json  - has_duplicate:  False
mf.filename:  test_fitter_coherent_purity_expected_results.json  - has_duplicate:  False
mf.filename:  test_fitter_coherent_purity_results.json  - has_duplicate:  False
mf.filename:  test_fitter_expected_results_1.json  - has_duplicate:  False
mf.filename:  test_fitter_expected_results_2.json  - has_duplicate:  

<ipython-input-28-b11954331418>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real_bugs["local_file_path"] = df_real_bugs.apply(


# Ad hoc inspection

In [11]:
SINGLE_REPO_FOLDER = "../repos/pennylane"
SINGLE_COMMIT = "504c58d74a6496d9dc72fe1667bac656f2ada18b"


single_repo = Repository(SINGLE_REPO_FOLDER, single=SINGLE_COMMIT)
for commit in single_repo.traverse_commits():
    #help(commit)
    print(commit.files)
    print(commit.parents)

2
['be468c9ee0567b3de91cc2ae597eacbea84ed56e', '137d17494f61e7490b7b3384cb595f13998fa5cf']
